# Librerias

In [6]:
import json
from spotipy.oauth2 import SpotifyOAuth
import spotipy

with open("D:/Academia/Ayudantia/Codigos/S6/spoty_JSON.json", "r") as f:
    creds = json.load(f)

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=creds["client_id"],
    client_secret=creds["client_secret"],
    redirect_uri=creds["redirect_uri"],
    scope=creds["scope"]
))

In [11]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from tqdm import tqdm

# Obtener credenciales

In [12]:
#ID del cliente.
client_id = creds["client_id"]

#Secret client.
client_secret = creds["client_secret"]

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)

#Cliente que sconsumirá la API.
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Datos de un artista

In [14]:
#info de Michael Jackson.
michael_info = sp.search(q='Michael Jackson', type='artist', limit=1)

michael_info

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=Michael%20Jackson&type=artist',
  'limit': 1,
  'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=Michael%20Jackson&type=artist',
  'offset': 0,
  'previous': None,
  'total': 856,
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3fMbdgg4jU18AjLCKBhRSm'},
    'followers': {'href': None, 'total': 36782269},
    'genres': [],
    'href': 'https://api.spotify.com/v1/artists/3fMbdgg4jU18AjLCKBhRSm',
    'id': '3fMbdgg4jU18AjLCKBhRSm',
    'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb997cc9a4aec335d46c9481fd',
      'height': 640,
      'width': 640},
     {'url': 'https://i.scdn.co/image/ab67616100005174997cc9a4aec335d46c9481fd',
      'height': 320,
      'width': 320},
     {'url': 'https://i.scdn.co/image/ab6761610000f178997cc9a4aec335d46c9481fd',
      'height': 160,
      'width': 160}],
    'name': 'Michael Jackson',
    'popularity': 87,
    'type':

Ahora accedemos a un elemento en particular del diccionario, en este caso, al número total de seguidores del grupo en la plataforma:

In [15]:
#Se imprime el número de seguidores de Michael Jackson.
michael_info['artists']['items'][0]['followers']['total']

36782269

In [16]:
#ID de MJ
MJ_id = result['artists']['items'][0]['id']

#Las 10 canciones más escuchadas de Michael Jackson
results = sp.artist_top_tracks(MJ_id)

for track in results['tracks']:
    print(track['name'])

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


Billie Jean
Beat It
Chicago
Smooth Criminal - 2012 Remaster
Rock with You - Single Version
The Way You Make Me Feel - 2012 Remaster
Thriller
Don't Stop 'Til You Get Enough
P.Y.T. (Pretty Young Thing)
Remember the Time


## Top 100 canciones de un género

Creamos una función para la extracción de 100 canciones de un género definido (parámetro de entrada), sus características, y ordena la información en un dataFrame de Pandas:

[Géneros de música](https://everynoise.com/everynoise1d.html)

In [21]:
#Obtiene n canciones de un género
def get_tracks_by_genre(genre, n=100, market=None):
    track_data = []
    limit = 50  #máximo permitido por Spotify

    for offset in range(0, n, limit):
        results = sp.search(q=f'genre:{genre}', type='track', limit=limit, offset=offset, market=market)
        track_ids = [track['id'] for track in results['tracks']['items']]
        
        if not track_ids:
            break

        tracks_info = sp.tracks(track_ids)

        for track in tracks_info['tracks']:
            track_data.append({
                'genre': genre,
                'name': track['name'],
                'artists_name': ', '.join([artist['name'] for artist in track['artists']]),
                'album_name': track['album']['name'],
                'album_release_date': track['album']['release_date'],
                'duration_ms': track['duration_ms'],
                'popularity': track['popularity'],
                'track_number': track['track_number'],
                'uri': track['uri']
            })

        #Si ya no devuelve canciones, rompemos el ciclo
        if len(results['tracks']['items']) < limit:
            break

    return pd.DataFrame(track_data)

In [22]:
df_pop = get_tracks_by_genre("dance pop", n=100, market="US")
df_pop.head()

Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


,genre,name,artists_name,album_name,album_release_date,duration_ms,popularity,track_number,uri
0,dance pop,We Like To Party! (The Vengabus),Vengaboys,The Party Album!,1999-06-04,222458,70,1,spotify:track:73ljrkSg4A0q3ByU0Cu7mw
1,dance pop,Duri Duri (Baila Baila),Click,Duri Duri,1987,294870,55,1,spotify:track:7ft4Kl0QcL5no8eTGtod39
2,dance pop,Brother Louie Mix '98 (feat. Eric Singleton) -...,"Modern Talking, Eric Singleton",Back For Good/2nd,1998-05-11,204306,69,15,spotify:track:2X7jfQnEMQYYmvkhr6mzS9
3,dance pop,Your Love - Radio Edit,Lime,Your Love,1981,242733,57,9,spotify:track:0wgXzjgHh13xFz9S92SZpN
4,dance pop,DON'T YOU WORRY,"Black Eyed Peas, Shakira, David Guetta",DON'T YOU WORRY,2022-06-17,196363,66,1,spotify:track:0gYXw7aPoybWFfB7btQ0eM


In [24]:
for i in sorted(df_pop["artists_name"].unique()):
  print(i)

ADÉLA
Alesso, Zara Larsson
Alok, Ella Eyre, Kenny Dope, Never Dull
Aqua
Ava Max
Avicii
Axwell /\ Ingrosso, Axwell, Sebastian Ingrosso
BLOND:ISH, Stevie Appleton
BUNT., Tom Odell
Baltimora
Black Eyed Peas, J Balvin
Black Eyed Peas, Shakira, David Guetta
CYRIL, maryjo
Calvin Harris, Disciples
Calvin Harris, Dua Lipa
Calvin Harris, Sam Smith
Calvin Harris, Sam Smith, Jessie Reyez
Cash Cash, Bebe Rexha
Click
DJ Snake, Bipolar Sunshine
David Guetta, Alphaville, Ava Max
David Guetta, Anne-Marie, Coi Leray
David Guetta, Bebe Rexha
David Guetta, Kelly Rowland
David Guetta, Sia
David Guetta, USHER
Eiffel 65, Gabry Ponte
Elton John, Dua Lipa, PNAU
Galantis
HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demon Hunters Cast, David Guetta
ILLENIUM, Norma Jean Martine
Icona Pop
Imagine Dragons, Broiler
James Hype, Sam Harper, Bobby Harvey
Joel Corry, Jax Jones, Charli xcx, Saweetie
Jonas Blue, Dakota
Kungs, Cookin' On 3 Burners
Kygo, Parson James
Kylie Minogue
Lady Gaga, Ariana Grande
Lilly Wood and The P

In [25]:
len(df_pop["artists_name"].unique())

74

In [31]:
#De esos artistas, cuántas veces aparecen en el top 100? (Solo los que aparecen 2 o más veces)
artist_counts = df_pop["artists_name"].value_counts().reset_index()
artist_counts.columns = ["artist", "count"]
artist_counts.sort_values("count", ascending=False)
artist_counts_filtered = artist_counts[artist_counts["count"] >= 2]
artist_counts_filtered

,artist,count
0,Avicii,7
1,Vengaboys,5
2,"Black Eyed Peas, J Balvin",3
3,"Lost Frequencies, Calum Scott",2
4,"Majestic, Boney M.",2
5,Regard,2
6,"David Guetta, Kelly Rowland",2
7,"Calvin Harris, Disciples",2
8,"Lilly Wood and The Prick, Robin Schulz",2
9,Kylie Minogue,2


Separar artistas para que estén de forma individual con `explode` y contabilizar bien sus apariciones

In [35]:
artist_counts_corrected = df_pop['artists_name'].str.split(', ').explode().value_counts().reset_index()
artist_counts_corrected.columns = ['artist', 'count']
artist_counts_corrected = artist_counts_corrected.sort_values("count", ascending=False)
artist_counts_filtered = artist_counts_corrected[artist_counts_corrected["count"] >= 2]
artist_counts_filtered

,artist,count
0,David Guetta,10
1,Avicii,7
2,Calvin Harris,6
3,Vengaboys,5
4,Dua Lipa,5
5,Ava Max,5
6,Black Eyed Peas,4
7,Lost Frequencies,3
8,Sam Smith,3
9,Robin Schulz,3


In [43]:
#¿Qué canciones de David Guetta	salen en el top 100?
canciones_david_guetta = df_pop[df_pop['artists_name'].str.contains("David Guetta")]
canciones_david_guetta

,genre,name,artists_name,album_name,album_release_date,duration_ms,popularity,track_number,uri
4,dance pop,DON'T YOU WORRY,"Black Eyed Peas, Shakira, David Guetta",DON'T YOU WORRY,2022-06-17,196363,66,1,spotify:track:0gYXw7aPoybWFfB7btQ0eM
14,dance pop,Without You (feat. Usher),"David Guetta, USHER",Nothing but the Beat (Ultimate Edition),2012-12-07,208133,76,4,spotify:track:3UN6cIn3VIyg0z1LCuFSum
15,dance pop,Beautiful People,"David Guetta, Sia",Beautiful People,2025-03-07,187086,83,1,spotify:track:4TwEdnSiTPDR1vg1QZ5K8W
20,dance pop,Baby Don't Hurt Me,"David Guetta, Anne-Marie, Coi Leray",Baby Don't Hurt Me,2023-04-06,140017,79,1,spotify:track:3BKD1PwArikchz2Zrlp1qi
23,dance pop,When Love Takes Over (feat. Kelly Rowland),"David Guetta, Kelly Rowland",One More Love,2010-11-22,191000,74,1,spotify:track:0GWGZA5mxo9nHuSyrj70uo
40,dance pop,Golden - David Guetta REM/X,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...",Golden (from the Netflix film KPop Demon Hunters),2025-06-19,176250,77,3,spotify:track:2nR5PWtpraC5cBZZ904oQr
58,dance pop,When Love Takes Over (feat. Kelly Rowland),"David Guetta, Kelly Rowland",One Love (Deluxe),2009-08-21,191000,59,5,spotify:track:77FULy278MulVvGWS8BfK7
59,dance pop,I'm Good (Blue),"David Guetta, Bebe Rexha",I'm Good (Blue),2022-08-26,175238,85,1,spotify:track:4uUG5RXrOk84mYEfFvj3cK
71,dance pop,Golden - David Guetta REM/X,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...",Golden [David Guetta REM/X (from the Netflix f...,2025-07-25,176250,72,1,spotify:track:5tVnzVWIKrmmuOPGbXUbT4
93,dance pop,Forever Young,"David Guetta, Alphaville, Ava Max",Forever Young,2024-10-18,156923,75,1,spotify:track:3bg2qahpZmsg5wV2EMPXIk


## Endpoints
* [Albums](https://developer.spotify.com/documentation/web-api/reference/get-an-album)
* [Artistas](https://developer.spotify.com/documentation/web-api/reference/get-an-artist)
* [Playlists](https://developer.spotify.com/documentation/web-api/reference/get-playlist)
* [Tracks](https://developer.spotify.com/documentation/web-api/reference/get-track)

# DataFrame

## Token de autorización

In [45]:
redurect_uri = "http://localhost:8888/callback"
scope = "user-library-read"

#Obtener el token de acceso
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redurect_uri,
    scope=scope
))

user_info = sp.current_user()
print("\n¡Autenticación exitosa!")
print(f"Hola, {user_info['display_name']} (ID: {user_info['id']})")

Couldn't read cache at: .cache
Using 'localhost' as a redirect URI is being deprecated. Use a loopback IP address such as 127.0.0.1 to ensure your app remains functional.


PermissionError: [WinError 10013] Intento de acceso a un socket no permitido por sus permisos de acceso